In [1]:
import sys
sys.path.insert(0, "/root/autodl-tmp/Code/RLHF")
sys.path.insert(0, "/Users/zeyesun/Documents/Code/RLHF")
sys.path.insert(0, "D:\\Code\\RLHF")
sys.path.insert(0, "/mnt/sfevol775196/sunzeye273/Code/chatgpt")
sys.path.insert(0, "/mnt/share-pa002-vol682688-prd/sunzeye273/Code/chatgpt")
sys.path.insert(0, "/mnt/pa002-28359-vol543625-private/Code/chatgpt")

import os, time, re, random, glob, json, jieba, copy
import numpy as np
import pandas as pd
from tqdm import tqdm
import torch
from transformers import (
    AutoModelForCausalLM,
    AutoModelForSeq2SeqLM,
    AutoModelForMultipleChoice,
    AutoTokenizer,
    Trainer,
    TrainingArguments,
    default_data_collator,
    TextGenerationPipeline
)

from src.models.reward import RewardModel

device="cuda:0" if torch.cuda.is_available() else "cpu"
from sys import platform
if platform == "linux" or platform == "linux2":
    # linux
    root = "/mnt/sfevol775196/sunzeye273/Data"
#     root = "/mnt/share-pa002-vol682688-prd/sunzeye273/Data"
#     root = "/mnt/pa002-28359-vol543625-private/Data"
#     root = "/root/autodl-tmp/Data"
elif platform == "darwin":
    # OS X
    root = "/Users/zeyesun/Documents/Data"
elif platform == "win32":
    # Windows...
    root = "D:\\Data"

2023-04-26 10:36:01.977849: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
model_name = "pangu-small"
# model_name = "glm-small"
# model_name = "chatglm-6B"
model_name_or_path = os.path.join(root, "models", model_name)

tokenizer = AutoTokenizer.from_pretrained(model_name_or_path, use_cache=False, trust_remote_code=True)
print(tokenizer.special_tokens_map)
print(tokenizer.all_special_ids)
print(
    f"unk: {tokenizer.unk_token_id}\n",
    f"pad: {tokenizer.pad_token_id}\n",
    f"bos: {tokenizer.bos_token_id}\n",
    f"eos: {tokenizer.eos_token_id}\n",
    f"sep: {tokenizer.sep_token_id}\n",
    f"mask: {tokenizer.mask_token_id}\n",
#     f"eop: {tokenizer.eop_token_id}\n"
#     f"sop: {tokenizer.sop_token_id}\n"
#     f"cls: {tokenizer.cls_token_id}"
) 

Explicitly passing a `revision` is encouraged when loading a model with custom code to ensure no malicious code has been contributed in a newer revision.
Building prefix dict from the default dictionary ...
Loading model from cache /var/folders/wf/12stcn_s6zq56j9h3fnkv5lm0000gn/T/jieba.cache
Loading model cost 0.481 seconds.
Prefix dict has been built successfully.


{'bos_token': '<s>', 'eos_token': '<eot>', 'unk_token': '<unk>', 'sep_token': '<sep>', 'pad_token': '<pad>'}
[1, 9, 0, 5, 6]
unk: 0
 pad: 6
 bos: 1
 eos: 9
 sep: 5
 mask: None



In [19]:
prompt = "你好，你是谁？"
prefix = "答:"
label = "我是ChatGPT"
max_length = 128

In [27]:
inputs = tokenizer(prompt, tokenizer.sep_token + prefix, max_length=max_length,
                                       truncation="longest_first", add_special_tokens=False,
                                       return_tensors="pt", return_token_type_ids=False)
print(inputs['input_ids'])
inputs = tokenizer(prompt, label, max_length=max_length,
                                       truncation="longest_first", 
                   padding="max_length",
#                    add_special_tokens=False,
                                       return_tensors="pt", 
                   return_token_type_ids=False)
print(inputs['input_ids'])

tensor([[5772,   10,   27,   16,  405,   28,    5, 3330,   17]])
tensor([[    1,  5772,    10,    27,    16,   405,    28,     5,    21,    16,
           542, 24396,  1122, 21723,     9,     6,     6,     6,     6,     6,
             6,     6,     6,     6,     6,     6,     6,     6,     6,     6,
             6,     6,     6,     6,     6,     6,     6,     6,     6,     6,
             6,     6,     6,     6,     6,     6,     6,     6,     6,     6,
             6,     6,     6,     6,     6,     6,     6,     6,     6,     6,
             6,     6,     6,     6,     6,     6,     6,     6,     6,     6,
             6,     6,     6,     6,     6,     6,     6,     6,     6,     6,
             6,     6,     6,     6,     6,     6,     6,     6,     6,     6,
             6,     6,     6,     6,     6,     6,     6,     6,     6,     6,
             6,     6,     6,     6,     6,     6,     6,     6,     6,     6,
             6,     6,     6,     6,     6,     6,     6,     6,  

In [35]:
chosen_id = inputs['input_ids'][0]
print(chosen_id)
seq_len = len(chosen_id)
c_inds = (chosen_id == tokenizer.pad_token_id).nonzero()
c_ind = c_inds[0].item() if len(c_inds) > 0 else seq_len

tensor([    1,  5772,    10,    27,    16,   405,    28,     5,    21,    16,
          542, 24396,  1122, 21723,     9,     6,     6,     6,     6,     6,
            6,     6,     6,     6,     6,     6,     6,     6,     6,     6,
            6,     6,     6,     6,     6,     6,     6,     6,     6,     6,
            6,     6,     6,     6,     6,     6,     6,     6,     6,     6,
            6,     6,     6,     6,     6,     6,     6,     6,     6,     6,
            6,     6,     6,     6,     6,     6,     6,     6,     6,     6,
            6,     6,     6,     6,     6,     6,     6,     6,     6,     6,
            6,     6,     6,     6,     6,     6,     6,     6,     6,     6,
            6,     6,     6,     6,     6,     6,     6,     6,     6,     6,
            6,     6,     6,     6,     6,     6,     6,     6,     6,     6,
            6,     6,     6,     6,     6,     6,     6,     6,     6,     6,
            6,     6,     6,     6,     6,     6,     6,     6])

In [ ]:
prompt_token = tokenizer(prompt, return_tensors="pt")
# prompt_token["input_ids"] = prompt_token["input_ids"]
# prompt_token["attention_mask"] = prompt_token["attention_mask"]
# print(prompt_token)
print(length - (max_length - 1))
for key_word in ["input_ids", "attention_mask"]:
    length = prompt_token[key_word].size()[-1]
    if length > max_seq_len:
        y = prompt_token[key_word].squeeze(0)[length - (max_length - 1):].flip(0)
    else:
        y = prompt_token[key_word].squeeze(0).flip(0)
    prompt_token[key_word] = y
# prompt_dataset.append(prompt_token)

In [15]:
# data = []
data.append([prompt_token["input_ids"], prompt_token["attention_mask"], tokenizer.pad_token_id])

In [16]:
from torch.nn.utils.rnn import pad_sequence
import torch.nn.functional as F

batch = {}

pad_token_id = data[-1][-1]

prompt = pad_sequence([f[0] for f in data],
                      padding_value=pad_token_id,
                      batch_first=True)
prompt_mask = pad_sequence([f[1] for f in data],
                           padding_value=0,
                           batch_first=True)

### make sure the final ouput is a seqence of 2**?
length = prompt.size()[-1]
pad_length = max_seq_len - length
if pad_length > 0:
    batch["prompt"] = F.pad(prompt,
                            pad=(pad_length, 0),
                            mode='constant',
                            value=pad_token_id)
    batch["prompt_att_mask"] = F.pad(prompt_mask,
                                     pad=(pad_length, 0),
                                     mode='constant',
                                     value=0)
else:
    batch["prompt"] = prompt
    batch["prompt_att_mask"] = prompt_mask
batch["prompt"] = batch["prompt"].flip(1)
batch["prompt_att_mask"] = batch["prompt_att_mask"].flip(1)
# return batch